In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
data = pd.DataFrame.from_csv('gene_high_throughput_sequencing.csv')
data.head()

,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Patient_id,,,,,,,,,,,,,,,,,,,,,
STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [26]:
print data.Diagnosis.value_counts()
wnorm = data[data.Diagnosis == 'normal'].values[:, 1:]
wearly = data[data.Diagnosis == 'early neoplasia'].values[:, 1:]
wcancer = data[data.Diagnosis == 'cancer'].values[:, 1:]
print wnorm.shape, wearly.shape, wcancer.shape

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64
(24, 15748) (25, 15748) (23, 15748)


In [28]:

%%time
norm_early = []
early_cancer = []
for i in xrange(data.shape[1] - 1):
    ne = stats.ttest_ind(wnorm[:, i], wearly[:, i], equal_var=False)
    norm_early.append(ne[1])
    ec = stats.ttest_ind(wearly[:, i], wcancer[:, i], equal_var=False)
    early_cancer.append(ec[1])


CPU times: user 7.64 s, sys: 48.3 ms, total: 7.69 s
Wall time: 7.74 s


In [29]:
early_cancer = np.array(early_cancer)
norm_early = np.array(norm_early)


In [30]:
ne_sum = (norm_early < 0.05).sum()
ec_sum = (early_cancer < 0.05).sum()
print ne_sum, ec_sum
with open('early_cancer.txt', 'w') as f:
    f.write(str(ec_sum))

with open('norm_early.txt', 'w') as f:
    f.write(str(ne_sum))


1575 3490


In [50]:
res_ne = smm.multipletests(norm_early, alpha = 0.025, method = 'holm')[0]
ne_sum = (np.max([wnorm.T[res_ne].mean(axis = 1) / wearly.T[res_ne].mean(axis = 1),
                 wearly.T[res_ne].mean(axis = 1) / wnorm.T[res_ne].mean(axis = 1)], axis= 0) > 1.5).sum()
rec_ec = smm.multipletests(early_cancer, alpha = 0.025, method = 'holm')[0]
ec_sum = (np.max([wearly.T[rec_ec].mean(axis = 1) / wcancer.T[rec_ec].mean(axis = 1),
                 wcancer.T[rec_ec].mean(axis = 1) / wearly.T[rec_ec].mean(axis = 1)], axis= 0) > 1.5).sum()


In [51]:
with open('early_cancer2.txt', 'w') as f:
    f.write(str(ec_sum))

with open('norm_early2.txt', 'w') as f:
    f.write(str(ne_sum))

In [101]:
def check_abs(non_rejected_ne, non_rejected_ec):
    n_means = wnorm.T[non_rejected_ne].mean(axis = 1)
    e_means = wearly.T[non_rejected_ne].mean(axis = 1)
    e2_means = wearly.T[non_rejected_ec].mean(axis = 1)
    c_means = wcancer.T[non_rejected_ec].mean(axis = 1)
    ne_sum = (np.max([n_means / e_means,
                      e_means / n_means],
                     axis= 0)
              > 1.5).sum()
    ec_sum = (np.max([e2_means / c_means,
                      c_means / e2_means],
                     axis= 0)
              > 1.5).sum()
    return ne_sum, ec_sum

In [102]:
res_ne = smm.multipletests(norm_early, alpha = 0.025, method = 'holm')[0]
rec_ec = smm.multipletests(early_cancer, alpha = 0.025, method = 'holm')[0]
print check_abs(res_ne, rec_ec)

(2, 77)


In [103]:
res_ne = smm.multipletests(norm_early, alpha = 0.025, method = 'fdr_bh')[0]
res_ec = smm.multipletests(early_cancer, alpha= 0.025, method = 'fdr_bh')[0]

ne_sum, ec_sum = check_abs(res_ne, rec_ec)
print ne_sum, ec_sum

e2_means = wearly.T[res_ec].mean(axis = 1)
c_means = wcancer.T[res_ec].mean(axis = 1)
ec_sum = np.max(np.abs(
                        [e2_means / c_means,
                         c_means / e2_means]),
                     axis= 0)

print ec_sum[ec_sum > 1.5].shape[0]

4 77
524


In [97]:
with open('early_cancer3.txt', 'w') as f:
    f.write('524')

with open('norm_early3.txt', 'w') as f:
    f.write(str(ne_sum))